# Tokenisation BPE (Byte Pair Encoding)

## Introduction

Bienvenue dans ce notebook sur la **tokenisation BPE** ! La tokenisation est la première étape cruciale dans le traitement du langage naturel. Elle consiste à découper un texte en unités plus petites appelées **tokens**.

### Pourquoi la tokenisation est-elle importante ?

Les modèles de langage comme GPT ne peuvent pas travailler directement avec du texte brut. Ils ont besoin de convertir le texte en nombres (IDs de tokens) pour effectuer des calculs mathématiques.

### Trois approches de tokenisation

1. **Tokenisation par mots** : Chaque mot = 1 token
   - ✅ Simple et intuitif
   - ❌ Vocabulaire énorme (millions de mots)
   - ❌ Ne gère pas les mots inconnus

2. **Tokenisation par caractères** : Chaque caractère = 1 token
   - ✅ Vocabulaire très petit (~100 caractères)
   - ✅ Gère tous les mots possibles
   - ❌ Séquences très longues
   - ❌ Perd la structure des mots

3. **Tokenisation par sous-mots (BPE)** : Compromis intelligent ⭐
   - ✅ Vocabulaire de taille raisonnable (10k-50k tokens)
   - ✅ Gère les mots inconnus
   - ✅ Capture les structures morphologiques
   - ✅ Utilisé par GPT, BERT, et la plupart des LLMs modernes

### L'algorithme BPE

BPE (Byte Pair Encoding) est un algorithme qui apprend automatiquement un vocabulaire de sous-mots en fusionnant itérativement les paires de caractères les plus fréquentes.

**Exemple simple** :
```
Corpus: "bas basse basses bassement"

Étape 1: Vocabulaire initial = caractères individuels
  ['b', 'a', 's', 'e', 'm', 'n', 't', '</w>']

Étape 2: Compter les paires adjacentes
  ('b', 'a'): 4 fois  ← Paire la plus fréquente!
  ('a', 's'): 4 fois
  ('s', 's'): 3 fois
  ...

Étape 3: Fusionner la paire la plus fréquente
  'b' + 'a' → 'ba'
  Vocabulaire: ['b', 'a', 's', 'e', 'm', 'n', 't', '</w>', 'ba']

Étape 4: Répéter jusqu'à atteindre la taille de vocabulaire souhaitée
  'ba' + 's' → 'bas'
  'bas' + 's' → 'bass'
  ...
```

### Objectifs de ce notebook

1. 📐 Comprendre le formalisme mathématique de BPE
2. 🔨 Implémenter BPE from scratch avec NumPy
3. 🚀 Utiliser une implémentation professionnelle optimisée
4. 📊 Visualiser la croissance du vocabulaire
5. 🧪 Tester l'encodage et le décodage

Commençons !

## 1. Formalisme Mathématique de BPE

### Formule de fréquence des paires

Pour une paire de tokens $(a, b)$, sa fréquence dans le corpus est :

$$f(a, b) = \sum_{i=1}^{N} \mathbb{1}[\text{seq}_i = (a, b)]$$

où :
- $N$ = nombre total de séquences dans le corpus
- $\mathbb{1}[\cdot]$ = fonction indicatrice (1 si vrai, 0 sinon)
- $\text{seq}_i$ = la $i$-ème séquence de tokens

### Critère de fusion

À chaque itération, on sélectionne la paire la plus fréquente :

$$(a^*, b^*) = \arg\max_{(a,b)} f(a, b)$$

Cette paire est ensuite fusionnée en un nouveau token : $t_{\text{new}} = ab$

### Algorithme complet

**Entrée** : Corpus $C$, nombre de fusions $M$

**Sortie** : Vocabulaire $V$, liste de fusions $\mathcal{M}$

1. **Initialisation** : $V \leftarrow \{\text{tous les caractères uniques dans } C\}$

2. **Pour** $m = 1$ **à** $M$ **faire** :
   
   a. Calculer $f(a, b)$ pour toutes les paires adjacentes
   
   b. $(a^*, b^*) \leftarrow \arg\max_{(a,b)} f(a, b)$
   
   c. $t_{\text{new}} \leftarrow a^* b^*$
   
   d. $V \leftarrow V \cup \{t_{\text{new}}\}$
   
   e. $\mathcal{M} \leftarrow \mathcal{M} \cup \{(a^*, b^*)\}$
   
   f. Remplacer toutes les occurrences de $(a^*, b^*)$ par $t_{\text{new}}$ dans $C$

3. **Retourner** $V, \mathcal{M}$

### Complexité

- **Temps** : $O(M \cdot |C| \cdot |V|)$ où $M$ = nombre de fusions, $|C|$ = taille du corpus, $|V|$ = taille du vocabulaire
- **Espace** : $O(|V| + M)$ pour stocker le vocabulaire et les fusions

## 2. Imports et Configuration

In [1]:
# Imports standard
import sys
from pathlib import Path

# Ajouter le répertoire src au path pour importer nos modules
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root / 'src'))

# Imports pour la visualisation
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

# Configuration de matplotlib pour de beaux graphiques
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("✅ Imports réussis!")
print(f"📁 Répertoire du projet: {project_root}")

✅ Imports réussis!
📁 Répertoire du projet: /home/frejus/Projects/laleye/Cours/NLP/mini-gpt-workshop


## 3. Corpus d'Entraînement

Nous allons utiliser un petit corpus français pour démontrer BPE.

In [4]:
# Corpus d'exemple en français
french_corpus = """
le chat est sur le tapis
le chien est dans le jardin
le chat mange la souris
le chien court dans le parc
la souris est petite
le tapis est grand
le jardin est beau
le parc est vert
le chat dort sur le tapis
le chien joue dans le jardin
""".strip()

# Le corpus est plutôt dans le fichier ../../data/raw/french_corpus.txt
french_corpus = (project_root / 'data' / 'raw' / 'french_corpus.txt').read_text(encoding='utf-8').strip()

print("📝 Corpus d'entraînement:")
print("=" * 60)
print(french_corpus)
print("=" * 60)
print(f"\n📊 Statistiques:")
print(f"  - Longueur: {len(french_corpus)} caractères")
print(f"  - Nombre de mots: {len(french_corpus.split())} mots")
print(f"  - Mots uniques: {len(set(french_corpus.split()))} mots")

# Afficher les mots les plus fréquents
word_counts = Counter(french_corpus.split())
print(f"\n🔝 Top 5 des mots les plus fréquents:")
for word, count in word_counts.most_common(5):
    print(f"  - '{word}': {count} fois")

📝 Corpus d'entraînement:
Le chat dort sur le canapé.
La tour Eiffel est un monument emblématique de Paris.
Les étudiants apprennent à construire des modèles de langage.
L'intelligence artificielle transforme notre société.
Le soleil brille dans le ciel bleu.
Les oiseaux chantent dans les arbres du jardin.
La France est connue pour sa gastronomie et sa culture.
Les transformers ont révolutionné le traitement du langage naturel.
Le chat qui était sur le tapis a mangé la souris.
Les réseaux de neurones profonds apprennent des représentations complexes.
Le chat dort sur le canapé.
La tour Eiffel est un monument emblématique de Paris.
Les étudiants apprennent à construire des modèles de langage.
L'intelligence artificielle transforme notre société.
Le soleil brille dans le ciel bleu.
Les oiseaux chantent dans les arbres du jardin.
La France est connue pour sa gastronomie et sa culture.
Les transformers ont révolutionné le traitement du langage naturel.
Le chat qui était sur le tapis a mangé

## 4. Implémentation From Scratch (NumPy)

### Objectif

Comprendre chaque étape de l'algorithme BPE en l'implémentant avec du Python pur et NumPy.

### Étapes de l'implémentation

1. **Initialisation** : Créer un vocabulaire de caractères
2. **Comptage** : Compter la fréquence de toutes les paires adjacentes
3. **Fusion** : Fusionner la paire la plus fréquente
4. **Itération** : Répéter jusqu'à atteindre le nombre de fusions souhaité
5. **Mapping** : Créer les mappings token ↔ ID

## SimpleBPETokenizer class

In [ ]:
from typing import List, Dict, Tuple, Set
from collections import Counter, defaultdict
import re

class SimpleBPETokenizer:
    """
    A from-scratch implementation of Byte Pair Encoding tokenizer.
    
    This implementation uses pure Python data structures to demonstrate
    the BPE algorithm step-by-step for educational purposes.
    
    Attributes:
        num_merges (int): Number of merge operations to perform
        vocab (Set[str]): Set of all tokens in the vocabulary
        merges (List[Tuple[str, str]]): Ordered list of merge operations
        token_to_id (Dict[str, int]): Mapping from token string to integer ID
        id_to_token (Dict[int, str]): Mapping from integer ID to token string
    """
    
    def __init__(self, num_merges: int = 1000):
        """
        Initialize the BPE tokenizer.
        
        Args:
            num_merges: Number of merge operations to perform during training
        """
        self.num_merges = num_merges
        self.vocab: Set[str] = set()
        self.merges: List[Tuple[str, str]] = []
        self.token_to_id: Dict[str, int] = {}
        self.id_to_token: Dict[int, str] = {}
    
    def train(self, corpus: str) -> None:
        """
        Entraîne le tokenizer BPE sur un corpus en apprenant les opérations de fusion.

        Cette méthode implémente l'algorithme BPE de base :
        1. Initialise le vocabulaire avec les caractères individuels
        2. Pour chaque itération de fusion :
           a. Compte la fréquence de toutes les paires de tokens adjacents
           b. Trouve la paire la plus fréquente
           c. Fusionne cette paire en un nouveau token
           d. Met à jour le vocabulaire et la liste des fusions

        Args:
            corpus: Texte d'entraînement sous forme d'une seule chaîne
        """
        # Step 1: Preprocess corpus - split into words
        # We use a simple regex to split on whitespace while preserving word boundaries
        words = re.findall(r'\S+', corpus)
        
        # Step 2: Initialize vocabulary with individual characters
        # Each word is represented as a list of characters with a special end-of-word marker
        # Example: "hello" -> ['h', 'e', 'l', 'l', 'o', '</w>']
        word_freqs = Counter(words)
        
        # Convert each word to a list of characters (with end-of-word marker)
        split_words = {}
        for word, freq in word_freqs.items():
            # Split word into characters and add end-of-word marker
            chars = list(word) + ['</w>']
            split_words[' '.join(chars)] = freq
        
        # Build initial vocabulary from all unique characters
        for word in split_words.keys():
            for char in word.split():
                self.vocab.add(char)
        
        print(f"Initial vocabulary size: {len(self.vocab)}")
        print(f"Initial vocabulary (first 20): {sorted(list(self.vocab))[:20]}")
        
        # Step 3: Perform iterative merging
        for merge_idx in range(self.num_merges):
            # Step 3a: Count frequency of all adjacent pairs
            pair_freqs = self._count_pair_frequencies(split_words)
            
            # Check if we have any pairs left to merge
            if not pair_freqs:
                print(f"No more pairs to merge. Stopping at iteration {merge_idx}")
                break
            
            # Step 3b: Find the most frequent pair
            # This implements: (a*, b*) = argmax f(a,b)
            best_pair = max(pair_freqs, key=pair_freqs.get)
            
            # Step 3c: Merge the best pair
            # Record this merge operation for later use during encoding
            self.merges.append(best_pair)
            
            # Create the new merged token
            new_token = ''.join(best_pair)
            self.vocab.add(new_token)
            
            # Step 3d: Update all words by applying this merge
            split_words = self._merge_pair_in_words(best_pair, split_words)
            
            # Print progress every 100 merges
            if (merge_idx + 1) % 100 == 0:
                print(f"Merge {merge_idx + 1}/{self.num_merges}: "
                      f"'{best_pair[0]}' + '{best_pair[1]}' -> '{new_token}' "
                      f"(freq: {pair_freqs[best_pair]})")
        
        # Step 4: Build token-to-ID and ID-to-token mappings
        # This ensures bidirectional mapping as required by Requirement 1.4
        sorted_vocab = sorted(list(self.vocab))
        for idx, token in enumerate(sorted_vocab):
            self.token_to_id[token] = idx
            self.id_to_token[idx] = token
        
        print(f"\nTraining complete!")
        print(f"Final vocabulary size: {len(self.vocab)}")
        print(f"Number of merges performed: {len(self.merges)}")
    
    def _count_pair_frequencies(self, split_words: Dict[str, int]) -> Dict[Tuple[str, str], int]:
        """
        Compte la fréquence de toutes les paires de tokens adjacents dans le vocabulaire courant.

        Cette fonction implémente la formule mathématique :
        f(a,b) = Σ 1[seq_i = (a,b)] pour toutes les séquences

        Args:
            split_words: Dictionnaire associant les séquences de tokens séparés par des espaces à leur fréquence

        Returns:
            Dictionnaire associant les paires de tokens à leur fréquence totale sur tous les mots
        """
        pair_freqs = defaultdict(int)
        
        for word, freq in split_words.items():
            tokens = word.split()
            
            # Count all adjacent pairs in this word
            for i in range(len(tokens) - 1):
                pair = (tokens[i], tokens[i + 1])
                pair_freqs[pair] += freq
        
        return dict(pair_freqs)
    
    def _merge_pair_in_words(self, pair: Tuple[str, str], 
                            split_words: Dict[str, int]) -> Dict[str, int]:
        """
        Apply a merge operation to all words in the vocabulary.
        
        This function replaces all occurrences of the specified pair with
        the merged token in all words.
        
        Args:
            pair: The pair of tokens to merge (a, b)
            split_words: Dictionary of space-separated token sequences
        
        Returns:
            Updated dictionary with the pair merged in all words
        """
        new_split_words = {}
        merged_token = ''.join(pair)
        
        for word, freq in split_words.items():
            tokens = word.split()
            
            # Apply the merge operation
            i = 0
            new_tokens = []
            while i < len(tokens):
                # Check if current position matches the pair to merge
                if i < len(tokens) - 1 and tokens[i] == pair[0] and tokens[i + 1] == pair[1]:
                    # Merge the pair
                    new_tokens.append(merged_token)
                    i += 2  # Skip both tokens in the pair
                else:
                    # Keep the token as is
                    new_tokens.append(tokens[i])
                    i += 1
            
            # Store the updated word
            new_word = ' '.join(new_tokens)
            new_split_words[new_word] = freq
        
        return new_split_words
    
    def encode(self, text: str) -> List[int]:
        """
        Encode text into a list of token IDs using the learned BPE vocabulary.
        
        This method applies the learned merge operations in order to tokenize
        the input text into subword units, then converts them to integer IDs.
        
        Process:
        1. Split text into words
        2. For each word, start with character-level tokens
        3. Apply all learned merges in order
        4. Convert final tokens to IDs
        
        Args:
            text: Input text to encode
        
        Returns:
            List of integer token IDs
        
        Requirements: 1.2 - Tokenize text into subword units
        """
        if not self.merges:
            raise ValueError("Tokenizer has not been trained. Call train() first.")
        
        # Split text into words
        words = re.findall(r'\S+', text)
        
        all_token_ids = []
        
        for word in words:
            # Start with character-level representation
            tokens = list(word) + ['</w>']
            
            # Apply all learned merges in order
            for merge_pair in self.merges:
                tokens = self._apply_merge_to_tokens(merge_pair, tokens)
            
            # Convert tokens to IDs
            for token in tokens:
                if token in self.token_to_id:
                    all_token_ids.append(self.token_to_id[token])
                else:
                    # Handle unknown tokens by encoding character-by-character
                    # This provides graceful fallback as mentioned in Requirement 1.5
                    for char in token:
                        if char in self.token_to_id:
                            all_token_ids.append(self.token_to_id[char])
        
        return all_token_ids
    
    def _apply_merge_to_tokens(self, pair: Tuple[str, str], tokens: List[str]) -> List[str]:
        """
        Apply a single merge operation to a list of tokens.
        
        Args:
            pair: The pair of tokens to merge
            tokens: List of current tokens
        
        Returns:
            Updated list of tokens with the merge applied
        """
        if len(tokens) < 2:
            return tokens
        
        merged_token = ''.join(pair)
        new_tokens = []
        i = 0
        
        while i < len(tokens):
            # Check if we can merge at this position
            if i < len(tokens) - 1 and tokens[i] == pair[0] and tokens[i + 1] == pair[1]:
                new_tokens.append(merged_token)
                i += 2
            else:
                new_tokens.append(tokens[i])
                i += 1
        
        return new_tokens
    
    def decode(self, token_ids: List[int]) -> str:
        """
        Decode a list of token IDs back into text.
        
        This method converts integer IDs back to their corresponding tokens,
        then reconstructs the original text by removing end-of-word markers
        and joining tokens appropriately.
        
        Args:
            token_ids: List of integer token IDs
        
        Returns:
            Decoded text string
        
        Requirements: 1.2 - Support round-trip encoding/decoding
        """
        if not self.id_to_token:
            raise ValueError("Tokenizer has not been trained. Call train() first.")
        
        # Convert IDs to tokens
        tokens = []
        for token_id in token_ids:
            if token_id in self.id_to_token:
                tokens.append(self.id_to_token[token_id])
            else:
                # Unknown ID - skip it
                continue
        
        # Reconstruct text by joining tokens and handling end-of-word markers
        text = ''.join(tokens)
        
        # Replace end-of-word markers with spaces
        text = text.replace('</w>', ' ')
        
        # Clean up extra spaces
        text = ' '.join(text.split())
        
        return text
    
    def get_vocab_size(self) -> int:
        """
        Get the size of the vocabulary.
        
        Returns:
            Number of tokens in the vocabulary
        """
        return len(self.vocab)
    
    def get_token_from_id(self, token_id: int) -> str:
        """
        Get the token string corresponding to a token ID.
        
        Args:
            token_id: Integer token ID
        
        Returns:
            Token string
        
        Requirements: 1.4 - Bidirectional token-ID mapping
        """
        return self.id_to_token.get(token_id, '<UNK>')
    
    def get_id_from_token(self, token: str) -> int:
        """
        Get the token ID corresponding to a token string.
        
        Args:
            token: Token string
        
        Returns:
            Integer token ID, or -1 if token not in vocabulary
        
        Requirements: 1.4 - Bidirectional token-ID mapping
        """
        return self.token_to_id.get(token, -1)
    
    def display_merge_examples(self, num_examples: int = 10) -> None:
        """
        Display examples of learned merge operations for pedagogical purposes.
        
        Args:
            num_examples: Number of merge examples to display
        """
        print(f"\nFirst {num_examples} merge operations:")
        print("-" * 60)
        for i, (token1, token2) in enumerate(self.merges[:num_examples]):
            merged = ''.join([token1, token2])
            print(f"{i+1}. '{token1}' + '{token2}' -> '{merged}'")
        print("-" * 60)


In [6]:
# Importer notre implémentation from scratch
from tokenization.bpe_from_scratch import SimpleBPETokenizer

print("✅ Module SimpleBPETokenizer importé avec succès!")

✅ Module SimpleBPETokenizer importé avec succès!


### 4.1 Entraînement du Tokenizer

In [7]:
# Créer et entraîner le tokenizer
print("🔨 Entraînement du tokenizer BPE from scratch...\n")

tokenizer_scratch = SimpleBPETokenizer(num_merges=30)
tokenizer_scratch.train(french_corpus)

print(f"\n✅ Entraînement terminé!")
print(f"📊 Taille finale du vocabulaire: {tokenizer_scratch.get_vocab_size()}")

🔨 Entraînement du tokenizer BPE from scratch...

Initial vocabulary size: 32
Initial vocabulary (first 20): ["'", '.', '</w>', 'E', 'F', 'L', 'P', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n']

Training complete!
Final vocabulary size: 62
Number of merges performed: 30

✅ Entraînement terminé!
📊 Taille finale du vocabulaire: 62


### 4.2 Visualisation des Fusions

Regardons les premières fusions apprises par l'algorithme.

In [ ]:
# Afficher les premières fusions
tokenizer_scratch.display_merge_examples(15)

# Analyser les fusions
print("\n📈 Analyse des fusions:")
print("  - Les premières fusions combinent des caractères fréquents")
print("  - On voit apparaître des préfixes communs: 'le', 'la', 'es', 'ch'")
print("  - Les fusions suivantes créent des mots complets ou des sous-mots")

### 4.3 Test d'Encodage et Décodage

Testons la propriété de **round-trip** : encoder puis décoder doit retourner le texte original.

In [ ]:
# Test 1: Phrase du corpus
test_text_1 = "le chat mange la souris"
print(f"📝 Texte original: '{test_text_1}'")

# Encoder
encoded_1 = tokenizer_scratch.encode(test_text_1)
print(f"🔢 IDs encodés: {encoded_1}")
print(f"   Nombre de tokens: {len(encoded_1)}")

# Décoder
decoded_1 = tokenizer_scratch.decode(encoded_1)
print(f"📝 Texte décodé: '{decoded_1}'")

# Vérifier le round-trip
round_trip_success = (test_text_1 == decoded_1)
print(f"✅ Round-trip réussi: {round_trip_success}")

# Afficher les tokens individuels
print(f"\n🔍 Tokens individuels:")
for i, token_id in enumerate(encoded_1):
    token = tokenizer_scratch.get_token_from_id(token_id)
    print(f"  {i}: ID={token_id:3d} → '{token}'")

In [ ]:
# Test 2: Phrase hors corpus (avec mots inconnus)
test_text_2 = "le robot intelligent"
print(f"\n📝 Texte avec mots inconnus: '{test_text_2}'")

encoded_2 = tokenizer_scratch.encode(test_text_2)
print(f"🔢 IDs encodés: {encoded_2}")
print(f"   Nombre de tokens: {len(encoded_2)}")

decoded_2 = tokenizer_scratch.decode(encoded_2)
print(f"📝 Texte décodé: '{decoded_2}'")

print(f"\n💡 Observation:")
print(f"  - 'le' est dans le vocabulaire → 1 token")
print(f"  - 'robot' et 'intelligent' sont inconnus → décomposés en caractères")
print(f"  - Le tokenizer gère gracieusement les mots inconnus (Requirement 1.5)")

### 4.4 Visualisation de la Croissance du Vocabulaire

Créons une visualisation montrant comment le vocabulaire grandit avec les fusions.

In [ ]:
# Entraîner plusieurs tokenizers avec différents nombres de fusions
merge_counts = [0, 5, 10, 20, 30, 50, 75, 100]
vocab_sizes = []

print("📊 Calcul de la croissance du vocabulaire...")
for num_merges in merge_counts:
    temp_tokenizer = SimpleBPETokenizer(num_merges=num_merges)
    temp_tokenizer.train(french_corpus)
    vocab_sizes.append(temp_tokenizer.get_vocab_size())
    print(f"  {num_merges:3d} fusions → {temp_tokenizer.get_vocab_size():3d} tokens")

# Créer le graphique
plt.figure(figsize=(12, 6))
plt.plot(merge_counts, vocab_sizes, marker='o', linewidth=2, markersize=8, color='#2E86AB')
plt.xlabel('Nombre de fusions', fontsize=13, fontweight='bold')
plt.ylabel('Taille du vocabulaire', fontsize=13, fontweight='bold')
plt.title('Croissance du Vocabulaire BPE', fontsize=15, fontweight='bold')
plt.grid(True, alpha=0.3)

# Annoter quelques points
for i in [0, 3, 5, 7]:
    plt.annotate(f'{vocab_sizes[i]}', 
                xy=(merge_counts[i], vocab_sizes[i]),
                xytext=(5, 5), textcoords='offset points',
                fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 Observations:")
print("  - Le vocabulaire commence avec les caractères uniques")
print("  - Chaque fusion ajoute exactement 1 nouveau token")
print("  - Croissance linéaire: vocab_size = initial_chars + num_merges")

## 5. Implémentation Professionnelle (Optimisée)

### Objectif

Utiliser une implémentation optimisée avec des fonctionnalités professionnelles :
- Gestion d'erreurs robuste
- Tokens spéciaux (PAD, UNK, BOS, EOS)
- Sauvegarde/chargement du vocabulaire
- Stratégies multiples pour les caractères inconnus
- Logging et debugging

In [ ]:
# Importer l'implémentation professionnelle
from tokenization.bpe_tokenizer import BPETokenizer

print("✅ Module BPETokenizer (professionnel) importé avec succès!")

### 5.1 Entraînement avec Tokens Spéciaux

In [ ]:
# Créer et entraîner le tokenizer professionnel
print("🚀 Entraînement du tokenizer BPE professionnel...\n")

tokenizer_pro = BPETokenizer(
    num_merges=30,
    use_special_tokens=True  # Activer les tokens spéciaux
)

tokenizer_pro.train(french_corpus, verbose=True)

print(f"\n✅ Entraînement terminé!")
print(f"📊 Taille du vocabulaire: {tokenizer_pro.get_vocab_size()}")

# Afficher les tokens spéciaux
special_tokens = tokenizer_pro.get_special_tokens()
print(f"\n🎯 Tokens spéciaux:")
for name, token_id in special_tokens.items():
    token_str = tokenizer_pro.get_token_from_id(token_id)
    print(f"  {name.upper():3s}: ID={token_id:3d} → '{token_str}'")

### 5.2 Comparaison des Stratégies pour Caractères Inconnus

Le tokenizer professionnel offre 3 stratégies pour gérer les caractères inconnus :
1. **char_fallback** : Décomposer en caractères (par défaut)
2. **unk_token** : Remplacer par le token UNK
3. **skip** : Ignorer les caractères inconnus

In [ ]:
# Texte avec caractères inconnus
unknown_text = "le chat xyz123 mange"
print(f"📝 Texte avec caractères inconnus: '{unknown_text}'\n")

# Stratégie 1: Character fallback (défaut)
print("1️⃣ Stratégie: char_fallback")
encoded_fallback = tokenizer_pro.encode(unknown_text, handle_unknown='char_fallback')
decoded_fallback = tokenizer_pro.decode(encoded_fallback)
print(f"   Encodé: {encoded_fallback}")
print(f"   Décodé: '{decoded_fallback}'")
print(f"   → Les caractères inconnus sont préservés\n")

# Stratégie 2: UNK token
print("2️⃣ Stratégie: unk_token")
encoded_unk = tokenizer_pro.encode(unknown_text, handle_unknown='unk_token')
decoded_unk = tokenizer_pro.decode(encoded_unk)
print(f"   Encodé: {encoded_unk}")
print(f"   Décodé: '{decoded_unk}'")
print(f"   → Les tokens inconnus sont remplacés par <UNK>\n")

# Stratégie 3: Skip
print("3️⃣ Stratégie: skip")
encoded_skip = tokenizer_pro.encode(unknown_text, handle_unknown='skip')
decoded_skip = tokenizer_pro.decode(encoded_skip)
print(f"   Encodé: {encoded_skip}")
print(f"   Décodé: '{decoded_skip}'")
print(f"   → Les tokens inconnus sont ignorés")

print("\n💡 Quelle stratégie choisir?")
print("  - char_fallback: Meilleur pour préserver l'information (recommandé)")
print("  - unk_token: Utile pour détecter les mots hors vocabulaire")
print("  - skip: Rarement utilisé, peut perdre de l'information")

### 5.3 Encodage avec Tokens Spéciaux BOS/EOS

Les tokens BOS (Beginning Of Sequence) et EOS (End Of Sequence) sont utilisés pour marquer le début et la fin d'une séquence.

In [ ]:
test_text = "le chat dort"
print(f"📝 Texte: '{test_text}'\n")

# Sans tokens spéciaux
print("Sans BOS/EOS:")
encoded_no_special = tokenizer_pro.encode(test_text, add_special_tokens=False)
print(f"  IDs: {encoded_no_special}")
print(f"  Tokens: {[tokenizer_pro.get_token_from_id(id) for id in encoded_no_special]}\n")

# Avec tokens spéciaux
print("Avec BOS/EOS:")
encoded_with_special = tokenizer_pro.encode(test_text, add_special_tokens=True)
print(f"  IDs: {encoded_with_special}")
print(f"  Tokens: {[tokenizer_pro.get_token_from_id(id) for id in encoded_with_special]}")

print("\n💡 Utilité des tokens BOS/EOS:")
print("  - BOS: Indique au modèle le début d'une nouvelle séquence")
print("  - EOS: Indique au modèle la fin de la séquence")
print("  - Essentiels pour la génération de texte (le modèle sait quand s'arrêter)")

### 5.4 Sauvegarde et Chargement du Vocabulaire

In [ ]:
# Sauvegarder le vocabulaire
vocab_path = project_root / 'data' / 'vocab' / 'bpe_vocab.json'
vocab_path.parent.mkdir(parents=True, exist_ok=True)

print(f"💾 Sauvegarde du vocabulaire...")
tokenizer_pro.save_vocabulary(vocab_path)
print(f"✅ Vocabulaire sauvegardé dans: {vocab_path}")

# Charger le vocabulaire dans un nouveau tokenizer
print(f"\n📂 Chargement du vocabulaire...")
tokenizer_loaded = BPETokenizer()
tokenizer_loaded.load_vocabulary(vocab_path)
print(f"✅ Vocabulaire chargé: {tokenizer_loaded}")

# Vérifier que ça fonctionne
test_text = "le chat mange"
encoded_original = tokenizer_pro.encode(test_text)
encoded_loaded = tokenizer_loaded.encode(test_text)

print(f"\n🔍 Vérification:")
print(f"  Texte: '{test_text}'")
print(f"  Encodé (original): {encoded_original}")
print(f"  Encodé (chargé):   {encoded_loaded}")
print(f"  Identiques: {encoded_original == encoded_loaded} ✅")

print("\n💡 Pourquoi sauvegarder le vocabulaire?")
print("  - Évite de ré-entraîner le tokenizer à chaque fois")
print("  - Garantit la cohérence entre entraînement et inférence")
print("  - Permet de partager le tokenizer avec d'autres")

## 6. Comparaison From Scratch vs Professionnel

Comparons les deux implémentations sur plusieurs critères.

In [ ]:
import time

# Corpus plus grand pour tester la performance
large_corpus = french_corpus * 10  # Répéter 10 fois

print("⚡ Test de performance\n")
print(f"Corpus: {len(large_corpus)} caractères, {len(large_corpus.split())} mots\n")

# Test 1: From Scratch
print("1️⃣ From Scratch:")
start = time.time()
tok_scratch = SimpleBPETokenizer(num_merges=50)
tok_scratch.train(large_corpus)
time_scratch = time.time() - start
print(f"   Temps d'entraînement: {time_scratch:.3f}s")

# Test 2: Professionnel
print("\n2️⃣ Professionnel:")
start = time.time()
tok_pro = BPETokenizer(num_merges=50)
tok_pro.train(large_corpus, verbose=False)
time_pro = time.time() - start
print(f"   Temps d'entraînement: {time_pro:.3f}s")

print(f"\n📊 Comparaison:")
print(f"   Ratio de vitesse: {time_scratch/time_pro:.2f}x")

# Comparaison des fonctionnalités
print("\n🔍 Comparaison des fonctionnalités:\n")
features = [
    ("Algorithme BPE de base", "✅", "✅"),
    ("Encodage/Décodage", "✅", "✅"),
    ("Mapping Token ↔ ID", "✅", "✅"),
    ("Tokens spéciaux (PAD, UNK, BOS, EOS)", "❌", "✅"),
    ("Gestion d'erreurs robuste", "❌", "✅"),
    ("Stratégies multiples pour inconnus", "❌", "✅"),
    ("Sauvegarde/Chargement vocabulaire", "❌", "✅"),
    ("Logging et debugging", "❌", "✅"),
    ("Validation des entrées", "❌", "✅"),
]

print(f"{'Fonctionnalité':<45} {'From Scratch':<15} {'Professionnel'}")
print("=" * 75)
for feature, scratch, pro in features:
    print(f"{feature:<45} {scratch:<15} {pro}")

print("\n💡 Conclusion:")
print("  - From Scratch: Excellent pour comprendre l'algorithme")
print("  - Professionnel: Prêt pour la production, robuste, complet")

## 7. Visualisation des Opérations de Fusion

Créons une visualisation interactive montrant comment les fusions transforment le texte.

In [ ]:
# Exemple simple pour visualiser les fusions
simple_corpus = "bas basse basses bassement"
print(f"📝 Corpus simple: '{simple_corpus}'\n")

# Entraîner avec seulement 5 fusions
viz_tokenizer = SimpleBPETokenizer(num_merges=5)
viz_tokenizer.train(simple_corpus)

print("\n🔍 Détail des 5 premières fusions:\n")
viz_tokenizer.display_merge_examples(5)

# Visualiser l'effet de chaque fusion
print("\n📊 Évolution du vocabulaire:\n")

# Vocabulaire initial (caractères)
initial_vocab = sorted([c for word in simple_corpus.split() for c in set(word)] + ['</w>'])
print(f"Étape 0 (Initial): {len(initial_vocab)} tokens")
print(f"  Vocabulaire: {initial_vocab}\n")

# Après chaque fusion
for i, (token1, token2) in enumerate(viz_tokenizer.merges, 1):
    merged = ''.join([token1, token2])
    print(f"Étape {i}: Fusion '{token1}' + '{token2}' → '{merged}'")
    print(f"  Nouveau vocabulaire: +1 token ('{merged}')\n")

In [ ]:
# Visualisation graphique de la croissance du vocabulaire
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Graphique 1: Nombre de tokens par mot
test_words = ["bas", "basse", "basses", "bassement"]
tokens_per_word = []

for word in test_words:
    encoded = viz_tokenizer.encode(word)
    tokens_per_word.append(len(encoded))

ax1.bar(test_words, tokens_per_word, color=['#2E86AB', '#A23B72', '#F18F01', '#C73E1D'])
ax1.set_xlabel('Mot', fontsize=12, fontweight='bold')
ax1.set_ylabel('Nombre de tokens', fontsize=12, fontweight='bold')
ax1.set_title('Tokenisation par Mot', fontsize=14, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)

# Annoter les valeurs
for i, (word, count) in enumerate(zip(test_words, tokens_per_word)):
    ax1.text(i, count + 0.1, str(count), ha='center', fontweight='bold')

# Graphique 2: Distribution des longueurs de tokens
token_lengths = [len(token) for token in viz_tokenizer.vocab]
length_counts = Counter(token_lengths)

lengths = sorted(length_counts.keys())
counts = [length_counts[l] for l in lengths]

ax2.bar(lengths, counts, color='#2E86AB')
ax2.set_xlabel('Longueur du token (caractères)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Nombre de tokens', fontsize=12, fontweight='bold')
ax2.set_title('Distribution des Longueurs de Tokens', fontsize=14, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Observations:")
print("  - Les mots fréquents sont tokenisés en moins de tokens")
print("  - Les mots rares sont décomposés en plus de sous-mots")
print("  - La plupart des tokens ont 1-3 caractères")

## 8. Exercices Pratiques

### Exercice 1: Entraîner votre propre tokenizer

Créez un tokenizer BPE sur un corpus de votre choix.

In [ ]:
# TODO: Créez votre propre corpus (au moins 5 phrases)
my_corpus = """
# Écrivez votre corpus ici
"""

# TODO: Entraînez un tokenizer avec 20 fusions
my_tokenizer = None  # Remplacez par votre code

# TODO: Affichez les 10 premières fusions

# TODO: Testez l'encodage et le décodage sur une phrase de votre corpus

### Exercice 2: Analyser l'impact du nombre de fusions

Comparez les tokenizers avec différents nombres de fusions.

In [ ]:
# TODO: Entraînez 3 tokenizers avec 10, 50, et 100 fusions

# TODO: Pour chaque tokenizer, encodez la phrase "le chat mange la souris"

# TODO: Comparez le nombre de tokens produits

# TODO: Quelle est la relation entre le nombre de fusions et le nombre de tokens?

### Exercice 3: Tester la robustesse

Testez comment le tokenizer gère différents types d'entrées.

In [ ]:
# TODO: Testez avec une chaîne vide

# TODO: Testez avec des caractères spéciaux (émojis, ponctuation)

# TODO: Testez avec des nombres

# TODO: Testez avec un texte très long (>1000 caractères)

# TODO: Documentez vos observations

## 9. Résumé et Points Clés

### Ce que nous avons appris

1. **Algorithme BPE** :
   - Commence avec un vocabulaire de caractères
   - Fusionne itérativement les paires les plus fréquentes
   - Crée un vocabulaire de sous-mots

2. **Formalisme mathématique** :
   - Fréquence des paires: $f(a, b) = \sum_{i=1}^{N} \mathbb{1}[\text{seq}_i = (a, b)]$
   - Critère de fusion: $(a^*, b^*) = \arg\max_{(a,b)} f(a, b)$

3. **Implémentation** :
   - From scratch: Comprendre chaque étape
   - Professionnelle: Robuste et prête pour la production

4. **Propriétés importantes** :
   - **Round-trip**: encoder puis décoder retourne le texte original
   - **Mapping bidirectionnel**: token ↔ ID
   - **Gestion des inconnus**: Fallback gracieux aux caractères

### Pourquoi BPE est utilisé dans les LLMs

- ✅ **Vocabulaire optimal**: Ni trop grand (mots), ni trop petit (caractères)
- ✅ **Gère les mots rares**: Décompose en sous-mots connus
- ✅ **Multilingue**: Fonctionne pour toutes les langues
- ✅ **Capture la morphologie**: Préfixes, suffixes, racines
- ✅ **Efficace**: Séquences de longueur raisonnable

### Prochaines étapes

Dans le prochain notebook, nous utiliserons ce tokenizer pour créer des **embeddings** - la représentation vectorielle des tokens qui sera l'entrée de notre modèle Mini-GPT.

### Ressources supplémentaires

- 📄 [Article original BPE](https://arxiv.org/abs/1508.07909) (Sennrich et al., 2016)
- 📄 [GPT-2 Tokenization](https://huggingface.co/docs/transformers/tokenizer_summary)
- 🎥 [Andrej Karpathy - Tokenization](https://www.youtube.com/watch?v=zduSFxRajkE)

---

**Félicitations ! Vous maîtrisez maintenant la tokenisation BPE ! 🎉**